# Calcul du tenseur d'Eshelby dans le cas des ellipses

## 0 Quelques fonctions utiles

In [1]:
import numpy as np
from numpy import pi
from numpy.random import random_sample
from numpy.linalg import inv
from numpy import dot
from scipy.spatial.transform import Rotation as R 
from classes import *

def Comp3333_to_66 (G) : 
    "Passe d'un tenseur de comportement  G 3x3x3x3 a une matrice de comportement F 6x6"
    F=np.zeros((6,6))
    for i in range(3):
        for j in range(3):
            F[i,j] = G[i,i,j,j]
            
        F[i,5]=(G[i,i,0,1]+G[i,i,1,0])/2.
        F[i,3]=(G[i,i,1,2]+G[i,i,2,1])/2. 
        F[i,4]=(G[i,i,2,0]+G[i,i,0,2])/2. 
        F[3,i]=(G[1,2,i,i]+G[2,1,i,i])/2. 
        F[4,i]=(G[0,2,i,i]+G[2,0,i,i])/2.
        F[5,i]=(G[0,1,i,i]+G[1,0,i,i])/2.

    F[4,4]=(G[0,2,0,2]+G[2,0,0,2]+G[0,2,2,0]+G[2,0,2,0])/4. 
    F[3,3]=(G[1,2,1,2]+G[2,1,1,2]+G[1,2,2,1]+G[2,1,2,1])/4.  
    F[5,5]=(G[0,1,0,1]+G[1,0,0,1]+G[0,1,1,0]+G[1,0,1,0])/4.  
    F[4,3]=(G[0,2,1,2]+G[2,0,1,2]+G[0,2,2,1]+G[2,0,2,1])/4.  
    F[4,5]=(G[0,2,1,0]+G[2,0,1,0]+G[0,2,0,1]+G[2,0,0,1])/4.  
    F[3,4]=(G[1,2,0,2]+G[2,1,0,2]+G[1,2,2,0]+G[2,1,2,0])/4.  
    F[5,4]=(G[0,1,0,2]+G[1,0,0,2]+G[0,1,2,0]+G[1,0,2,0])/4.  
    F[3,5]=(G[1,2,1,0]+G[2,1,1,0]+G[1,2,0,1]+G[2,1,0,1])/4.   
    F[5,3]=(G[0,1,1,2]+G[1,0,1,2]+G[0,1,2,1]+G[1,0,2,1])/4. 
    
    return F

def Comp66_to_3333(F) : 
    'Passe d une matrice F 6x6 à un tenseur G 3x3x3x3'
    G = np.zeros((3,3,3,3))
    for i in range(3) :
        for j in range(3) :
            G[i,i,j,j]=F[i,j]
       
        G[i,i,0,1]=F[i,5]
        G[i,i,1,2]=F[i,3]
        G[i,i,2,0]=F[i,4]
        G[0,2,i,i]=F[4,i]
        G[1,2,i,i]=F[3,i]
        G[0,1,i,i]=F[5,i]
        G[i,i,1,0]=F[i,5]
        G[i,i,2,1]=F[i,3]
        G[i,i,0,2]=F[i,4]
        G[2,0,i,i]=F[4,i]
        G[2,1,i,i]=F[3,i]
        G[1,0,i,i]=F[5,i]
        
    G[0,1,0,1]=F[5,5]
    G[0,1,0,2]=F[5,4]
    G[0,1,1,0]=F[5,5]
    G[0,1,1,2]=F[5,3] 
    G[0,1,2,0]=F[5,4]
    G[0,1,2,1]=F[5,3]

    G[0,2,0,1]=F[4,5]
    G[0,2,0,2]=F[4,4]
    G[0,2,1,0]=F[4,5]
    G[0,2,1,2]=F[4,3] 
    G[0,2,2,0]=F[4,4]
    G[0,2,2,1]=F[4,3]

    G[1,0,0,1]=F[5,5]
    G[1,0,0,2]=F[5,4]
    G[1,0,1,0]=F[5,5]
    G[1,0,1,2]=F[5,3] 
    G[1,0,2,0]=F[5,4]
    G[1,0,2,1]=F[5,3]

    G[1,2,0,2]=F[3,4]
    G[1,2,1,0]=F[3,5]
    G[1,2,1,2]=F[3,3] 
    G[1,2,2,0]=F[3,4]
    G[1,2,2,1]=F[3,3]

    G[2,0,0,1]=F[4,5]
    G[2,0,0,2]=F[4,4]
    G[2,0,1,0]=F[4,5]
    G[2,0,1,2]=F[4,3] 
    G[2,0,2,0]=F[4,4]
    G[2,0,2,1]=F[4,3]

    G[2,1,0,1]=F[3,5]
    G[2,1,0,2]=F[3,4]
    G[2,1,1,0]=F[3,5]
    G[2,1,1,2]=F[3,3] 
    G[2,1,2,0]=F[3,4]
    G[2,1,2,1]=F[3,3]
 
    return G 

def Rotation_angles():
    theta = 2*pi*np.random.random()
    psi = 2*pi*np.random.random()
    phi = np.arccos(np.random.random())
    return (phi,theta,psi)


def Matrice_rotation(psi,phi,theta) : 
    'Crée une matrice de rotation 3x3 à partir des trois angles d euler'
    Q = np.zeros((3,3))
    
    Q[0,0]=cos(psi)*cos(theta)-cos(phi)*sin(theta)*sin(psi)
    Q[0,1]=sin(theta)*cos(psi)+cos(phi)*sin(psi)*cos(theta)
    Q[0,2]=sin(phi)*sin(psi)
    Q[1,0]=-sin(psi)*cos(theta)-sin(theta)*cos(phi)*cos(psi)
    Q[1,1]=cos(psi)*cos(phi)*cos(theta)-sin(theta)*sin(psi)
    Q[1,2]=cos(psi)*sin(phi)
    Q[2,0]=sin(phi)*sin(theta)
    Q[2,1]=-sin(phi)*cos(theta)
    Q[2,2]=cos(phi)
    
    for i in range(3) : 
        for j in range(3):
            if (abs(Q[i,j]) < 10**-6 ) :
                Q[i,j] = 0
            
    return Q

def Rotation_tenseur(S,phi,theta,psi) : 
    ' Renvoie la rotation du tenseur S par les 3 angles d Euler'
    B = np.zeros((3,3,3,3))
    R = Matrice_rotation(psi,phi,theta)
    for  i in range(3) : 
        for  j in range(i+1):
            for  k in range(3):
                for  l in range(k+1):
                    for  m in range(3):
                        for  n in range(3):
                            for  ll in range(3):
                                for  kk in range(3):
                                    B[i,j,k,l] += R[i,m]*R[j,n]*R[k,ll]*R[l,kk]*S[m,n,ll,kk]
                                    B[i,j,l,k] = B[i,j,k,l]
                                    B[j,i,k,l] = B[i,j,k,l]
                                    B[j,i,l,k] = B[i,j,k,l]
    return B


def Matrice_Souplesse_Isotrope(E,nu) :
    'Renvoie la matrice de souplesse d un matériau isotrope'
    S = np.zeros((6,6))
    S[0,0]=1./E
    S[1,1]=1./E
    S[2,2]=1./E

    S[3,3]=2.*(1+nu)/E
    S[4,4]=2.*(1+nu)/E
    S[5,5]=2.*(1+nu)/E

    S[0,1]=-nu/E
    S[0,2]=-nu/E
    S[1,2]=-nu/E
    S[1,0]=-nu/E
    S[2,1]=-nu/E
    S[2,0]=-nu/E
    
    return S
    

def Young_isotrope(S) : 
    return 3/(S[0,0]+S[1,1]+S[2,2])

def nu_isotrope(S) : 
    Ey = 1/S[1,1]
    Ez = 1/S[2,2]
    return - 1/3 * (Ey*S[0,1] + Ez*S[0,2] + Ez*S[1,2])
    
def Young_anisotrope(C) : 
    return C[0,0],C[1,1],C[2,2]


def Compute_with_permutation(a,I,II,nu) : 
    S = np.zeros((3,3,3,3))
    
    for i in range(3) :
        S[i,i,i,i] = 3*a[i]**2*II[i][i] / (8*pi*(1-nu)) + I[i] * (1-2*nu)/(8*pi*(1-nu))
        j = (i+1)%3
        S[i,i,j,j] = a[j]**2*II[i][j]/(8*pi*(1-nu)) -  I[i] * (1-2*nu)/(8*pi*(1-nu))
        S[i,j,i,j] = (a[i]**2+a[j]**2)*II[i][j]/(16*pi*(1-nu)) + (1-2*nu)/(16*pi*(1-nu))*(I[i]+I[j])
        k = (i+2)%3
        S[i,i,k,k] = a[k]**2*II[i][k]/(8*pi*(1-nu)) -  I[i] * (1-2*nu)/(8*pi*(1-nu))
        S[i,k,i,k] = (a[i]**2+a[k]**2)*II[i][k]/(16*pi*(1-nu)) + (1-2*nu)/(16*pi*(1-nu))*(I[i]+I[k])
        
    return S

def Cyclic_permutation(S) : 
    for i in range(3) : 
        for j in range(3) :
            for k in range(3) : 
                for l in range(3) : 
                    val = non_zeros(S,i,j,k,l)
                    S[i,j,k,l] = val
                    S[i,j,l,k] = val
                    S[j,i,k,l] = val
                    S[j,i,l,k] = val
    return S
                    

def non_zeros(S,i,j,k,l) : 
    if S[i,j,k,l] != 0 : 
        return S[i,j,k,l]
    if S[i,j,l,k] != 0 : 
        return S[i,j,l,k]
    if S[j,i,k,l] != 0 : 
        return S[j,i,k,l]
    if S[j,i,l,k] != 0 : 
        return S[j,i,l,k]
    return 0

def clear_matrix3 (C,k) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j,k]<10**-8 : 
                C[i,j,k] = 0
                
def clear_matrix2 (C) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j]<10**-8 : 
                C[i,j] = 0
    

In [2]:
E = 1
nu = 0.3
S = Matrice_Souplesse_Isotrope(E,nu)
nu = nu_isotrope(S)
print(S)
print(nu)

[[ 1.  -0.3 -0.3  0.   0.   0. ]
 [-0.3  1.  -0.3  0.   0.   0. ]
 [-0.3 -0.3  1.   0.   0.   0. ]
 [ 0.   0.   0.   2.6  0.   0. ]
 [ 0.   0.   0.   0.   2.6  0. ]
 [ 0.   0.   0.   0.   0.   2.6]]
0.29999999999999993


## I Calcul du tenseur d'Eshelby : Cas Isotropes

### 1) Cas sphéroïdaux

Définition du tenseur d'Eshelby dans le repère de l'Ellipse

In [3]:
from classes import *
import numpy as np
         
         
def Eshelby_tensor_sph(microstructure) :
    
    # Recupération des paramètres geometriques de l'ellipse 
    dict_inclusions = microstructure.dict_inclusions
    inclusion = list(dict_inclusions.keys())[0] #Inclusion unique ici
    Cm = microstructure.matrix_behavior
    nu = Cm['nu']
        
    if  inclusion.aspect_ratio == 1 :
        'Sphere'
        a0 = 1
        a1 = 1
        a2 = inclusion.aspect_ratio
        
        I0 = 4*pi/3
        I1 = I0
        I2 = I0

        I00 = 4*pi/(5*a0**2)
        I01 = I00
        I02 = I00
        I10 = I00
        I11 = I00
        I12 = I00
        I20 = I00
        I21 = I00
        I22 = I00

        
    if  inclusion.aspect_ratio < 1 :
        'Oblate'             
        a0 = 1
        a1 = 1
        a2 = inclusion.aspect_ratio
        g = a2/a0 # g<1
        
        I0 = 2*pi*a0**2*a2/np.power(a0**2-a2**2,3/2) * (np.arccos(g)-g*np.sqrt(1-g**2) )
        I1 = I0
        I2 = 4*pi-2*I0

        I02 = (I0-I2)/(a2**2-a0**2)
        I01 = pi/a0**2 - I02/4
        I00 = I01

        I12 = I02
        I11 = I01
        I10 = I01
         
        I20 = I02
        I21 = I12
        I22 = 1/3 * (4*pi/a2**2-2*I02)

    if  inclusion.aspect_ratio > 1 :
        'Prolate'
        a0 = inclusion.aspect_ratio
        a1 = 1
        a2 = 1       
        g = a0/a2 # g>1
        
        I1 = 2*pi*a0*a2**2/np.power(a0**2-a2**2,3/2) * (g*np.sqrt(g**2-1)-np.arccosh(g) )
        I2 = I1
        I0 = 4*pi-2*I1
         
       
        I01 = (I1-I0)/(a0**2-a1**2)
        I02 = I01
        I00 = 1/3 * (4*pi/a0**2-2*I01)
        
        I10 = I01
        I12 = pi/a1**2 - I01/4        
        I11 = I12
         
        I20 = I02
        I21 = I12
        I22 = I12

        
    a = [a0,a1,a2]
    I = [I0,I1,I2]
    II =[[I00,I01,I02] , [I10, I11 , I12] , [I20,I21,I22]]
    S = Compute_with_permutation(a,I,II,nu)
    S = Cyclic_permutation(S)
    
    return S
    
    
    

### Tests 

In [4]:
# Paramètres 
f_inclusion = 0.1
Em,num = 1,0.0
Ef,nuf = 10,0.0
inclusion_behavior = {"E":Ef, "nu":nuf}        
matrix_behavior = {"E":Em, "nu":num} 

## SPHERE
inclusion = Inclusion(0 ,inclusion_behavior,1)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
S =Eshelby_tensor_sph(microstructure)
C1 = Comp3333_to_66(S)
print(C1)
print(S[0,1,0,1])


## OBLATE
inclusion = Inclusion(1 ,inclusion_behavior,0.2)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
S =Eshelby_tensor_sph(microstructure)
C1 = Comp3333_to_66(S)
#print(C1)


## PROLATE
inclusion = Inclusion(2 ,inclusion_behavior,1.2)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
S =Eshelby_tensor_sph(microstructure)
C1 = Comp3333_to_66(S)
print(C1)


[[ 0.46666667 -0.06666667 -0.06666667  0.          0.          0.        ]
 [-0.06666667  0.46666667 -0.06666667  0.          0.          0.        ]
 [-0.06666667 -0.06666667  0.46666667  0.          0.          0.        ]
 [ 0.          0.          0.          0.26666667  0.          0.        ]
 [ 0.          0.          0.          0.          0.26666667  0.        ]
 [ 0.          0.          0.          0.          0.          0.26666667]]
0.26666666666666666
[[ 0.41132641 -0.06259942 -0.06259942  0.          0.          0.        ]
 [-0.06259942  0.49311983 -0.0735842   0.          0.          0.        ]
 [-0.06259942 -0.0735842   0.49311983  0.          0.          0.        ]
 [ 0.          0.          0.          0.28335202  0.          0.        ]
 [ 0.          0.          0.          0.          0.25893247  0.        ]
 [ 0.          0.          0.          0.          0.          0.25893247]]


### 2) Ellipsoïdes isotropes

Référence : [Toshio Mura, 1987 ,  Micromechanics of defects in solids ]

Par. 7.3.6, p.91

In [5]:
from classes import *
import numpy as np
from scipy.integrate import quad
from numpy import pi,sqrt



def delta(A,s) :
    a1,a2,a3 = A
    return sqrt((a1**2+s)*(a2**2+s)*(a3**2+s))
    
def Eshelby_tensor_ell(A,Cm) :
    
    # Recupération des paramètres geometriques de l'ellipse 
    a0,a1,a2 = A
    nu = Cm['nu']
    #compatible = check_hypothesis(A)
    compatible = True
    if not compatible : 
        raise NameError("Unable to treat spheroidal cases")

    # Calcul du tenseur d'Eshelby formulation sans relation d'ordre entre a1,a2,a3
    # Il y a une autre formulation avec a1>a2>a3 qui évite les intégrales généralisées (interessant pour les complexes)
    
    I0 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a0**2+x) * delta(A,x)) , 0 , np.inf)[0]
    I1 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a1**2+x) * delta(A,x)) , 0 , np.inf)[0]
    I2 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a2**2+x) * delta(A,x)) , 0 , np.inf)[0]
    
    I00 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a0**2+x)**2 * delta(A,x)) , 0 , np.inf)[0]  
    I01 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a0**2+x)*(a1**2+x) * delta(A,x)) , 0 , np.inf)[0]
    I02 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a2**2+x)*(a0**2+x) * delta(A,x)) , 0 , np.inf)[0]

    I10 = I01
    I11 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a1**2+x)**2 * delta(A,x)) , 0 , np.inf)[0] 
    I12 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a2**2+x)*(a1**2+x) * delta(A,x)) , 0 , np.inf)[0]
    
    I20 = I02
    I21 = I12
    I22 = 2*pi*a0*a1*a2*quad(lambda x : 1/((a2**2+x)**2 * delta(A,x)) , 0 , np.inf)[0] 
    
    
    I = [I0,I1,I2]
    II =[[I00,I01,I02] , [I10, I11 , I12] , [I20,I21,I22]]
    S = Compute_with_permutation(A,I,II,nu)
    S = Cyclic_permutation(S)
    
    return S

### TEST

In [6]:
Cm = {'nu':0.0}
A = 1,1,1
S = Eshelby_tensor_ell(A,Cm)
C1 = Comp3333_to_66(S)
print(S)
print(C1)

[[[[ 0.46666667  0.          0.        ]
   [ 0.         -0.06666667  0.        ]
   [ 0.          0.         -0.06666667]]

  [[ 0.          0.26666667  0.        ]
   [ 0.26666667  0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.26666667]
   [ 0.          0.          0.        ]
   [ 0.26666667  0.          0.        ]]]


 [[[ 0.          0.26666667  0.        ]
   [ 0.26666667  0.          0.        ]
   [ 0.          0.          0.        ]]

  [[-0.06666667  0.          0.        ]
   [ 0.          0.46666667  0.        ]
   [ 0.          0.         -0.06666667]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.26666667]
   [ 0.          0.26666667  0.        ]]]


 [[[ 0.          0.          0.26666667]
   [ 0.          0.          0.        ]
   [ 0.26666667  0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.26666667]
   [ 0.          0.26666667  0.        

### 3) Modèle autocohérent pour un seul type d'ellipse isotrope :

Une seule forme d'ellipse à orientations multiples

In [ ]:
from scipy.spatial.transform import Rotation as Rot

def compute_h_behavior(microstructure,n_renforts):
    
    n_renforts = 10     # paramètre non physique qui permet de forcer lisotropie
    n_pas = 100         # pas du modèle autocohérent
    precision = 10**-3  # précision désirée dans l'algorithme du point fixe
    
    #compatible = self.check_hypothesis(microstructure)
    compatible = True
    if not compatible:
        raise NameError("The microstructure does not match the model hypothesis") 
    

    dict_inclusions = microstructure.dict_inclusions
    inclusion = list(dict_inclusions.keys())[0] ## ici on a un seul type d'ellipse
    A = [1,1,1] ## dimension de l'ellipse
    
    f  = dict_inclusions[inclusion] ## Fraction volumique des renforts
    
    Cm = microstructure.matrix_behavior ## Matrice 66
    Cf = inclusion.behavior ## Matrice 66
    # Esh = Eshelby_tensor_sph(microstructure) ## 1 Matrice 66, il y en aura autant que d'ellipses de forme différentes

    # Création des matrices de comportement
    Sm = Matrice_Souplesse_Isotrope(Cm['E'],Cm['nu'])
    Cm = inv(Sm)
    Sf = Matrice_Souplesse_Isotrope(Cf['E'],Cf['nu'])
    Cf = inv(Sf)    
    Id = np.identity(6)
    
    
    # Boucle d'incrémentation du pas
    for i in range(n_pas) : 
        f_pas = f*i/n_pas
        f_1_renfort = f_pas/n_renforts
        
        # Algorithme du point fixe : Ch=f(Ch)
        Cp = Cm
        Sp = Sm
        convergence = 2
        younglast = Young_isotrope(Sm)
        while convergence>precision :             
            W = np.zeros((6,6)) # Matrice des contributions de l'inclusion dans Ch
            nu = nu_isotrope(Sp)
            Esh = Comp3333_to_66(Eshelby_tensor_ell(A,{'nu':nu}))
            Aesh = inv(Id + np.dot(np.dot(Esh,Sm),Cf-Cm))
            V6 = np.dot(Cm-Cf,Aesh)
            V3 = Comp66_to_3333(V6)
            # Ajout des contribution de chaque renfort en fonction de son orientation
            for i in range(n_renforts) : 
                phi,theta,psi = Rot.random().as_euler('zxy', degrees=True)
                V3R = Rotation_tenseur(V3,phi,theta,psi)
                V = Comp3333_to_66(V3R)
                W += f_1_renfort * V
                
            Ch = Cm + W
            # Actualisation du matériau homogénéisé
            Cp = Ch
            Sp = inv(Cp)
            
            # Test de sortie
            young = Young_isotrope(Sp)
            nu = nu_isotrope(Sp)
            convergence = abs(young-younglast)
            younglast = young
            
            # Forçage de la matrice en matrice isotrope
            Sp = Matrice_Souplesse_Isotrope(young,nu)
            Cp = inv(Sp)
            
    print(Young_anisotrope(Ch))
    return Ch
                
    

# Paramètres 
f_inclusion = 0.1
Em,num = 1,0.3
Ef,nuf = 10,0.3
inclusion_behavior = {"E":Ef, "nu":nuf}        
matrix_behavior = {"E":Em, "nu":num} 
nrenforts = 200

## SPHERE
inclusion = Inclusion(0 ,inclusion_behavior,1)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
print(compute_h_behavior(microstructure,nrenforts))

### Test

In [8]:
# Paramètres 
f_inclusion = 0.1
Em,num = 1,0.3
Ef,nuf = 10,0.3
inclusion_behavior = {"E":Ef, "nu":nuf}        
matrix_behavior = {"E":Em, "nu":num} 
nrenforts = 200

## SPHERE
inclusion = Inclusion(0 ,inclusion_behavior,1)
microstructure = Microstructure(matrix_behavior,{inclusion : f_inclusion})
print(compute_h_behavior(microstructure,nrenforts))

(1.3461538461538463, 1.346153846153846, 1.346153846153846)
[[1.34615385e+00 5.76923077e-01 5.76923077e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.76923077e-01 1.34615385e+00 5.76923077e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.76923077e-01 5.76923077e-01 1.34615385e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.85604641e-01
  5.32516780e-03 5.38259159e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.32516780e-03
  2.92236438e-01 3.68178853e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.38259159e-04
  3.68178853e-04 2.91535856e-01]]


# Calcul du tenseur d'Eshelby dans le cas général

In [7]:
def Eshelby_tensor(Axis,Em,nu) : 
    
    Sm = Matrice_Souplesse_Isotrope(Em,nu)
    Cm = inv(Sm)
    Cm3 = Comp66_to_3333(Cm)
    a0,a1,a2 = Axis
    IJV = np.array([[0,0],[1,1],[2,2],[1,2],[0,2],[0,1]])
    Nit = 40
    Ntop = Nit
    Mtop = Nit
    dphi = pi/(Ntop-1)
    dtheta = pi/(Ntop-1)
    A = np.zeros((6,6))
    B = np.zeros((6,6,Mtop))
    G = np.zeros((6,6,Ntop))
    E = np.zeros((6,6))
    
    # Integration de la fonction de green sur la demi ellipsoïde
    for m in range(Mtop) : 
        phi = m*dphi
        for n in range(Ntop) : 
            theta = n*dtheta
            X = np.array([sin(theta)*cos(phi)/a0 , sin(theta)*sin(phi)/a1 , cos(theta)/a2])
            CXX = np.zeros((3,3))
            for i in range(3) :
                for j in range(3) :
                    for k in range(3) : 
                        for l in range(3) :
                            CXX[i,k] += Cm3[i,j,k,l]*X[j]*X[l]
            CXX = inv(CXX)
            for i in range(6) :
                for j in range(6) :                     
                    I1 = IJV[i,0]
                    J1 = IJV[j,0]
                    I2 = IJV[i,1]
                    J2 = IJV[j,1]
                    G[i,j,n] = 0.5 * sin(theta) * (CXX[I1,J1]*X[I2]*X[J2] + CXX[I2,J1]*X[I1]*X[J2] + CXX[I1,J2]*X[I2]*X[J1] + CXX[I2,J2]*X[I1]*X[J1])
        
        
        B[:,:,m] = 0.5 * dtheta * (G[:,:,0]+G[:,:,Ntop-1])
        for i in range(1,Ntop-1) : 
            B[:,:,m] +=  dtheta * G[:,:,i]

    A = 0.5*(B[:,:,0]+B[:,:,Ntop-1])* dphi/(4*pi)
    for i in range(1,Ntop-1) : 
         A += B[:,:,i]* dphi/(4*pi)  
    
    for i in range(6) : 
        for j in range(6) : 
            E[i,j]=A[i,0]*Cm[0,j]+A[i,1]*Cm[1,j]+A[i,2]*Cm[2,j] + 4* (A[i,3]*Cm[3,j]+A[i,4]*Cm[4,j]+A[i,5]*Cm[5,j]) 
    
    return E

In [11]:
E = Eshelby_tensor((1,1,1),1,0)
clear_matrix2(E)
print(E)

[[0.46666702 0.         0.         0.         0.         0.        ]
 [0.         0.46666702 0.         0.         0.         0.        ]
 [0.         0.         0.46612587 0.         0.         0.        ]
 [0.         0.         0.         0.53279201 0.         0.        ]
 [0.         0.         0.         0.         0.53279201 0.        ]
 [0.         0.         0.         0.         0.         0.53333369]]


In [16]:
from scipy.spatial.transform import Rotation as Rot
        
def compute_h_behavior(A):
    
    n_renforts = 100     # paramètre non physique qui permet de forcer lisotropie
    n_pas = 2         # pas du modèle autocohérent
    precision = 10**-4  # précision désirée dans l'algorithme du point fixe

    # Création des matrices de comportement
    Em = 1
    num = 0
    Ef = 10
    nuf = 0
    f = 0.02
    
    Sm = Matrice_Souplesse_Isotrope(Em,num)
    Cm = inv(Sm)
    Sf = Matrice_Souplesse_Isotrope(Ef,nuf)
    Cf = inv(Sf)    
    Id = np.identity(6)

    # Boucle d'incrémentation du pas
    for i in range(n_pas) : 
        f_pas = f*i/n_pas
        f_1_renfort = f_pas/n_renforts
        
        # Algorithme du point fixe : Ch=f(Ch) pour f fixé
        Cp = Cm
        Sp = Sm
        convergence = 2
        
        Eh = Young_isotrope(Sm)
        nuh = nu_isotrope(Sm)
        
        print('f_pas : ',f_pas)
        print('convergence',convergence)
        while convergence>precision :   
            W = np.zeros((6,6)) # Matrice des contributions de l'inclusion dans Ch
            Esh = Eshelby_tensor(A,Eh,nuh)
            Aesh = inv(Id + np.matmul(Esh,np.matmul(Sm,Cf-Cm)))
            V6 = np.dot(Cf-Cm,Aesh)
            V3 = Comp66_to_3333(V6)
            # Ajout des contribution de chaque renfort en fonction de son orientation
            for i in range(n_renforts) : 
                theta,phi,psi = Rot.random().as_euler('zxy', degrees=False)
                #phi,theta,psi = Rotation_angles()
                V3R = Rotation_tenseur(V3,phi,theta,psi)
                V = Comp3333_to_66(V3R)
                W += f_1_renfort * V
            Ch = Cm + W
            # Actualisation du matériau homogénéisé
            Cp = Ch
            Sp = inv(Cp)
            
            # Test de sortie
            E = Young_isotrope(Sp)
            nu = nu_isotrope(Sp)
            convergence = abs(E-Eh)
            print('convergence',convergence)
            Eh = E
            nuh = nu            
            
            # Forçage de la matrice en matrice isotrope
            Sp = Matrice_Souplesse_Isotrope(Eh,nuh)
            Cp = inv(Sp)
            
    return Cp

In [17]:
A = (1.9,1,1.2)
print(compute_h_behavior(A),)

f_pas :  0.0
convergence 2
convergence 0.0
f_pas :  0.01
convergence 2
convergence 0.01845179640678829
convergence 2.9707203608175803e-06
[[1.01846643 0.00243964 0.00243964 0.         0.         0.        ]
 [0.00243964 1.01846643 0.00243964 0.         0.         0.        ]
 [0.00243964 0.00243964 1.01846643 0.         0.         0.        ]
 [0.         0.         0.         0.50801339 0.         0.        ]
 [0.         0.         0.         0.         0.50801339 0.        ]
 [0.         0.         0.         0.         0.         0.50801339]]
